In [ ]:
from os import environ as env
env['TF_CPP_MIN_LOG_LEVEL'] = '2'               # hide info & warnings
env['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'       # grow GPU memory as needed

import tensorflow as tf
import nstesia as nst

In [ ]:
def load_style_models(styles):
    return [
        nst.johnson_2016.StyleTransferModel.from_saved(f'saved/{style}')
        for style in styles
    ]

In [ ]:
def content_style_grid(image_files, styles):
    content_images = [ nst.io.load_image(file) for file in image_files ]
    style_models = load_style_models(styles)

    images = [None] + [model.style_image for model in style_models]
    for content_image in content_images:
        images += [content_image] + [
            model(content_image) for model in style_models
        ]

    grid_image = nst.image.grid(images, ncols=len(styles)+1)
    return tf.keras.utils.array_to_img(tf.squeeze(grid_image,axis=0))

In [ ]:
content_style_grid(
    ['img/content/sunflower.jpg', 'img/content/bochum.jpg'],
    styles=['candy', 'cubism', 'udnie'],
)

In [ ]:
content_style_grid(
    ['img/content/brad.jpg', 'img/content/karya.jpg'],
    styles=['bathing', 'delaunay', 'scream'],
)

In [ ]:
content_image = nst.io.load_image('img/content/chicago.jpg')
style_models = load_style_models(['candy', 'candy-bn'])

grid_image = nst.image.grid([model(content_image) for model in style_models])
tf.keras.utils.array_to_img(tf.squeeze(grid_image,axis=0))